In [140]:
from instruments_recognition.importing_data import *
from instruments_recognition.plotting import *
#import pymir.AudioFile

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['clf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [192]:
### Import transformed data (features) from file

instrument1 = "banjo"
instrument2 = "viola"


energylbl1af = open('transformed_data/' + instrument1 + '_energylbla', 'rb')
energylbl1rf = open('transformed_data/' + instrument1 + '_energylblr', 'rb')
energylbl2af = open('transformed_data/' + instrument2 + '_energylbla', 'rb')
energylbl2rf = open('transformed_data/' + instrument2 + '_energylblr', 'rb')

energylbl1a_withmore = np.load(energylbl1af)
energylbl1r_withmore = np.load(energylbl1rf)
energylbl2a_withmore = np.load(energylbl2af)
energylbl2r_withmore = np.load(energylbl2rf)

#print(energylbl1a_withmore[0])

In [210]:
### LEARN 1


howmanyharmonics = 5

# forget the frequencies
# and forget the first volume which is always 1
def only_volumes(energylbl) :
    return list(map(lambda l : l[1][1:howmanyharmonics+1], energylbl))

# take only non nan from a list
def cleannan(l):
    return [x for x in l if not isnan(x)]

# input: list of lists
# output: only lists that do not contain any nans
def cleannan2(l):
    return [x for x in l if np.isfinite(x).all()]


energylbl1a = only_volumes(energylbl1a_withmore)
energylbl1r = only_volumes(energylbl1r_withmore)
energylbl2a = only_volumes(energylbl2a_withmore)
energylbl2r = only_volumes(energylbl2r_withmore)

# how many harmonics do we have
print(str(len(energylbl1a_withmore[1][1][1:])) + " harmonics avaiable")

# we use attack and release and the difference of volume between the two
#energylbl1 = [ (l1 + l2 + v) for l1, l2, v in zip(energylbl1a,energylbl1r,vd1)]
#energylbl2 = [ (l1 + l2 + v) for l1, l2, v in zip(energylbl2a,energylbl2r,vd2)]

# but in the attack we sometimes fail to recognize the tonic (it is mainly confused with the octave)
# so we can use the release and the difference of volume between attack and release
#energylbl1 = [ (l1 + v) for l1, l2, v in zip(energylbl1a,energylbl1r,vd1)]
#energylbl2 = [ (l1 + v) for l1, l2, v in zip(energylbl2a,energylbl2r,vd2)]

# *but* it seems to work better with just l2
energylbl1 = energylbl1r
energylbl2 = energylbl2r


volumes1 = cleannan2(energylbl1)
volumes2 = cleannan2(energylbl2)
#shuffle the input so we train on a random set of the input (the shuffle is done in place )
random.seed(1)
random.shuffle(volumes1)
random.shuffle(volumes2)




#nthvolume = [ map(lambda l : l[n], only_volumes) for n in range(0,num_harmonics) ]
#
#nthvolumecleaned = map(cleannan, nthvolume)
#
#averages = map(average, nthvolumecleaned)
#
#minusaverages = [ map(lambda m : m-averages[n], nthvolumecleaned[n]) for n in range(0,num_harmonics) ]
#
#variances = map(rms, minusaverages)
#
#variances, averages

#print(vd1[2])

8 harmonics avaiable


In [211]:
### LEARN 2

from sklearn import neighbors, svm

#fractiontrain = 9./10.
fractiontrain = 5./6.
#fractiontrain = 4./5.
#fractiontrain = 1./100.

#training examples
trainvolumes1 = volumes1[:int(len(volumes1)*fractiontrain)]
trainvolumes2 = volumes2[:int(len(volumes2)*fractiontrain)]

howmanytrain1 = len(trainvolumes1)
howmanytrain2 = len(trainvolumes2)
howmanytrain = howmanytrain1 + howmanytrain2

#testing examples
testvolumes1 = volumes1[int(len(volumes1)*fractiontrain):len(volumes1)]
testvolumes2 = volumes2[int(len(volumes2)*fractiontrain):len(volumes2)]

howmanytest1 = len(testvolumes1)
howmanytest2 = len(testvolumes2)


training_ex = trainvolumes1 + trainvolumes2
labels = [ 1 for i in range(0,howmanytrain1) ] + [ 0 for i in range(0,howmanytrain2) ]


#########################

## assert: there are no testing samples in the training set

def is_in(e,l):
    res = False
    for v in l:
        if all([f1 == f2 for f1, f2 in zip(e,v)]) :
            res = True
    return res
            
#testing: is_in([1.],[[0.],[0.,4,3],[1.]])
    
alltests = testvolumes1 + testvolumes2

datasnooping = any([ is_in(test, training_ex) for test in alltests ])
if datasnooping :
    print("WARNING: you are testing on the training set")
print("doing data snooping? " + str(datasnooping))

doing data snooping? False


In [212]:
### Learning algorithm

## SVN does not seem to work
#clf = svm.SVC(kernel='linear')
#clf.fit(training_ex, labels) 


# nearest neighboor
n_neighbors = 5
clf = neighbors.KNeighborsClassifier(n_neighbors)
clf.fit(training_ex, labels) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [213]:
### COMPUTING ERROR

print("training with: " + str(howmanytrain))
print("testing with: " + str(howmanytest1 + howmanytest2))

goods1 = list(clf.predict(testvolumes1))
goods2 = [ 1-x for x in clf.predict(testvolumes2) ]

error = 1. - np.average(goods1+goods2)

print("error: " + str(error))


training with: 682
testing with: 138
error: 0.0942028985507
